In [2]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import function_library_assignment_1 as fnc

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4

In [3]:
n_bus = 24
S_base_3ph = 100
gen_data = fnc.read_data('gen_data')
system_demand = fnc.read_data('system_demand')['System Demand']
load_distribution = fnc.read_data('load_distribution')
gen_data = fnc.read_data('gen_data')
gen_costs = fnc.read_data('gen_costs')
line_data = fnc.read_data('line_data')
branch_matrix = fnc.read_data('branch_matrix')
wind_data = fnc.read_data('wind_data')

In [4]:
t = 0 #hour
load = np.zeros(n_bus)

#Saving the load for each bus in a numpy array accounting for the system load destribution
for n in load_distribution['Node'].unique():
    load[n-1] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * system_demand[t] / S_base_3ph #per-unitized load - remember that the data is not 0-indexed but the arrays are

load

/var/folders/bc/ynvgg9rn14v5sl_jcp8x8_s40000gn/T/ipykernel_50938/208695547.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  load[n-1] = load_distribution.loc[load_distribution['Node'] == n, r'% of system load'] / 100 * system_demand[t] / S_base_3ph #per-unitized load - remember that the data is not 0-indexed but the arrays are


array([0.6748173 , 0.6037839 , 1.11877605, 0.4617171 , 0.44395875,
       0.8524008 , 0.7813674 , 1.065501  , 1.08325935, 1.2075678 ,
       0.        , 0.        , 1.65152655, 1.2075678 , 1.97117685,
       0.62154225, 0.        , 2.07772695, 1.1365344 , 0.79912575,
       0.        , 0.        , 0.        , 0.        ])

In [5]:
gens_map, wf_map = fnc.mapping_dictionaries(gen_data)

gens_map.get(14) #example: get the generator indices at bus 14 (15 in the assignment formulation)

[4, 5]

# Task 2 - Augmented Lagrangian Relaxation

### 24 subproblems one for each node (not clear how to tackle previous iterations)

In [ ]:
# What values of gamma, tolerance, max iterations, and initial values of lambda, mu_upper, and mu_lower should be used?
gamma = 385 # penalty parameter
alpha = 1 # penalty parameter
beta = 1 # penalty parameter
tolerance = 0.01 # convergence tolerance
lambda_ini = 2346 # initial value of lambda
mu_upper_ini = -22 # initial value of mu_upper
mu_lower_ini = 22 # initial value of mu_lower
max_iterations = 100 # maximum number of iterations
finished = False


theta_i = np.zeros(n_bus) #initial values of theta
theta_i_prev = theta_i.copy() #previous values of theta
pg_i = np.ones(len(gen_data.index)) #initial values of pg
pg_i_prev = pg_i.copy() #previous values of pg
pw_i = np.zeros(len(wind_data.columns)) #initial values of pw
pw_i_prev = pw_i.copy() #previous values of pw

lambda_i = np.ones(n_bus) * lambda_ini
mu_upper_i = np.ones((n_bus, n_bus)) * mu_upper_ini
mu_lower_i = np.ones((n_bus, n_bus)) * mu_lower_ini

delta_upper_prev = np.zeros((n_bus, n_bus))
delta_lower_prev = np.zeros((n_bus, n_bus))

# lists to store error values
x = []
y_lambda = []
y_mu_upper = []
y_mu_lower = []

iter = 0

direction = gb.GRB.MINIMIZE #Min / Max


while (not finished):
    print('\n-----------------------------------------------\n')
    print("Iteration %d..." % iter)
    for i in range(n_bus): #a subproblem for each node i
        m = gb.Model()
        m.setParam('OutputFlag', False)
        
        num_gen_bus = len(gens_map.get(i)) #number of generators at node n
        num_wf_bus = len(wf_map.get(i)) #number of wind farms at node n

        # Add variables
        if num_gen_bus > 0:
            p_G = m.addVars(num_gen_bus, lb=0, ub=gb.GRB.INFINITY, name="P_G") #Note: This is in per unit
        else:
            p_G = []

        if num_wf_bus > 0:
            p_W = m.addVars(num_wf_bus, lb=0, ub=gb.GRB.INFINITY, name="P_W")
        else:
            p_W = []

        theta = m.addVar(lb=-gb.GRB.INFINITY, ub=gb.GRB.INFINITY, name="theta") #one theta for each subproblem
        delta_upper = m.addVars(n_bus, lb=0, ub=gb.GRB.INFINITY, name="delta_upper")
        delta_lower = m.addVars(n_bus, lb=0, ub=gb.GRB.INFINITY, name="delta_lower")
        
        #Since the "previous iteration penalties" for the constraint violations of the ADMM are scalars, we calculate them outside the objective function definition
        a_violation_scalar = 0
        b_violation_scalar = 0
        c_violation_scalar = 0

        for j in range(n_bus):
            if j != i:
                a_violation_scalar += (sum(pg_i_prev[g] for g in gens_map.get(j)) + sum(pw_i_prev[w] for w in wf_map.get(j)) - load[j] - theta_i_prev[j] * branch_matrix[j,j] - 
                                        sum(theta_i_prev[k] * branch_matrix[j,k] for k in range(n_bus) if k != j))
                
                b_violation_scalar += (sum(branch_matrix[j,k] * (theta_i_prev[j] - theta_i_prev[k]) - 
                                                  line_data.loc[(line_data['From'] == j + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum() + delta_upper_prev[j,k] for k in range(n_bus) if k != j))

                c_violation_scalar += (sum(-1 * branch_matrix[j,k] * (theta_i_prev[j] - theta_i_prev[k]) - 
                                                  line_data.loc[(line_data['From'] == j + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum() + delta_lower_prev[j,k] for k in range(n_bus) if k != j))


        obj = (gb.quicksum(gen_costs['C (DKK/MWh)'][k] * p_G[k] * S_base_3ph for k in range(len(p_G))) 
               + lambda_i[i] * ((gb.quicksum(p_G[g] for g in range(num_gen_bus)) + gb.quicksum(p_W[w] for w in range(num_wf_bus)) - load[i] - theta * branch_matrix[i,i] - 
                                gb.quicksum(theta_i[k] * branch_matrix[i,k] for k in range(n_bus) if k != i)))

                + gamma/2 * ((a_violation_scalar
                             + gb.quicksum(p_G[g] for g in range(num_gen_bus)) + gb.quicksum(p_W[w] for w in range(num_wf_bus)) - load[i] - theta * branch_matrix[i,i] - 
                                gb.quicksum(theta_i[k] * branch_matrix[i,k] for k in range(n_bus) if k != i)) ** 2)

                + gb.quicksum((mu_upper_i[i,k] * (branch_matrix[i,k] * (theta - theta_i[k]) - line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum())) for k in range(n_bus) if k != i)

                + alpha/2 * ((b_violation_scalar
                             + gb.quicksum((mu_upper_i[i,k] * (branch_matrix[i,k] * (theta - theta_i[k]) - line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum() + delta_upper[k])) for k in range(n_bus) if k != i)
                            ) ** 2)

                + gb.quicksum((-1 * mu_lower_i[i,k] * (branch_matrix[i,k] * (theta - theta_i[k]) - line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum())) for k in range(n_bus) if k != i)

                + beta/2 * ((c_violation_scalar
                             + gb.quicksum(-1 * (mu_lower_i[i,k] * (branch_matrix[i,k] * (theta - theta_i[k]) - line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum() + delta_lower[k])) for k in range(n_bus) if k != i)
                            ) ** 2)
                                
        )
        
        m.setObjective(obj, direction)

        # These constraints are the same for all subproblems

        gen_lims = [gen_data['P max MW'].iloc[g] / S_base_3ph for g in gens_map.get(i)]
        wf_lims = [wind_data.iloc[t, w] / S_base_3ph for w in wf_map.get(i)]
        
        m.addConstrs(p_G[g] <= gen_lims[g] for g in range(len(gen_lims))) #if the generator is on (s_G[i] = 1), then the limit is p_max
        m.addConstrs(p_G[g] >= 0 for g in range(len(p_G))) #P_min from the system should be disregarded to avoid having a mixed integer program
        m.addConstrs(p_W[w] <= wf_lims[w] for w in range(len(wf_lims))) #Maximum wind power is the per-unitized output of the non-curtailed wind farm in the hour t
        m.addConstrs(p_W[w] >= 0 for w in range(len(p_W))) #Wind farms can be curtailed
        if i == 0:
            m.addConstr(theta == 0)
        
        m.update()
        m.display()

        # solve the model
        m.optimize()

        if m.status == gb.GRB.OPTIMAL:
            # save variables
            theta_i[i] = theta.x

            node_gens = gens_map.get(i)
            pg_i[node_gens] = np.array([p_G[l].x for l in range(len(p_G))])

            node_wf = wf_map.get(i)
            pw_i[node_wf] = np.array([p_W[l].x for l in range(len(p_W))])

            delta_upper_prev[i,:] = np.array([delta_upper[i].x for i in range(len(delta_upper))])
            delta_lower_prev[i,:] = np.array([delta_lower[i].x for i in range(len(delta_lower))])
        else:
            print("Optimization for node %d was NOT successful." % i)

        # Dispose of the Gurobi model to release resources
        m.dispose()

    # save the previous values of theta, pg, and pw for next iteration
    theta_i_prev = theta_i.copy()
    pg_i_prev = pg_i.copy()
    pw_i_prev = pw_i.copy()

    # check convergence
    lambda_next = lambda_i + gamma * sum(sum(pg_i[g] for g in gens_map.get(i)) + sum(pw_i[w] for w in wf_map.get(i)) - load[i] - theta_i[i] * branch_matrix[i,i] - 
                                sum(theta_i[k] * branch_matrix[i,k] for k in range(n_bus) if k != i) for i in range(n_bus))
    
    mu_upper_next = mu_upper_i + alpha * sum(sum((branch_matrix[i,k] * (theta_i[i] - theta_i[k]) - 
                                                                            line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for k in range(n_bus) if k != i) for i in range(n_bus))

    mu_lower_next = mu_lower_i + beta * sum(sum(-1 * (branch_matrix[i,k] * (theta_i[i] - theta_i[k]) - 
                                                                            line_data.loc[(line_data['From'] == i + 1) & (line_data['To'] == k + 1), 'Capacity pu'].sum()) for k in range(n_bus) if k != i) for i in range(n_bus))

    #Force the indices of mu where there is no branch to be zero - just like the branch matrix... Otherwise it may cause issues. 
    # These mu values technically don't exist.
    for i in range(n_bus):
        for k in range(n_bus):
            if branch_matrix[i,k] == 0:
                mu_upper_next[i,k] = 0
                mu_lower_next[i,k] = 0

    # avoid division by zero
    try:
        error_lambda = np.abs(lambda_next - lambda_i) / np.abs(lambda_i)
    except: 
        error_lambda = 1
    try:
        error_mu_upper = np.abs(mu_upper_next - mu_upper_i) / np.abs(mu_upper_i)
    except: 
        error_mu_upper = 1
    try:
        error_mu_lower = np.abs(mu_lower_next - mu_lower_i) / np.abs(mu_lower_i)
    except:
        error_mu_lower = 1

    
    print("Avg. lambda: ", np.nanmean(lambda_next), " Error: ", np.nanmean(error_lambda))
    print("Avg. mu upper: ", np.nanmean(mu_upper_next), " Error: ", np.nanmean(error_mu_upper))
    print("Avg. mu lower: ", np.nanmean(mu_lower_next), " Error: ", np.nanmean(error_mu_lower))
    print('Demand - Generation: ', np.sum(load) - np.sum(pg_i) - np.sum(pw_i))
    # store values for later plotting
    x.append(iter)
    y_lambda.append(abs(np.nanmean(error_lambda)))
    y_mu_upper.append(abs(np.nanmean(error_mu_upper)))
    y_mu_lower.append(abs(np.nanmean(error_mu_lower)))


    if (np.all(error_lambda <= tolerance)) and (np.all(error_mu_upper <= tolerance)) and (np.all(error_mu_lower <= tolerance)):
        finished = True
        print("\nConvergence achieved! Converged at iteration %d." % iter)

    elif iter >= max_iterations:
        finished = True
        print("\nMaximum number of iterations reached. Convergence not achieved.")

    else:
        lambda_i = lambda_next.copy()
        mu_upper_i = mu_upper_next.copy()
        mu_lower_i = mu_lower_next.copy()
        iter += 1

In [ ]:
# plot the error values vs iterations
fig, ax = plt.subplots(figsize=(16 ,5 ) , dpi=300) # Create the figure
ax.plot(x, y_lambda, label='lambda', alpha = 0.5, color='darkgreen')
ax.plot(x, y_mu_upper, label='mu_upper', alpha=0.5, color='darkred')
ax.plot(x, y_mu_lower, label='mu_lower', alpha=0.5, color='darkblue')
ax.set_xlabel('Iteration')
ax.set_ylabel('Error')
ax.set_title('ADMM Convergence')
ax.set_ylim(0, 2)
ax.legend()
# place legend in top right corner
ax.legend(loc='upper right',bbox_to_anchor=(1.15, 1.0))
# show final error values on plot
ax.text(x[-1], y_mu_lower[-1], str(y_mu_lower[-1]))
# save plot
plt.savefig('Figures/ADMM_convergence.png',bbox_inches='tight')
plt.show()